# TO DO:
- How to handle NaN in encoder? (drop before classification)
- Mess with classifier parameters
- test classifier
- how to handle repeats?
- change order of operations so that slave_indexes match unclassified_data...how to do this?


In [1]:
import numpy as np
import pandas as pd
import exploringShipLogbooks.wordCount as wc
from sklearn import preprocessing
from sklearn.naive_bayes import MultinomialNB
import collections

from exploringShipLogbooks.basic_utils import isolate_training_data
from exploringShipLogbooks.basic_utils import extract_logbook_data
from exploringShipLogbooks.basic_utils import isolate_columns
from exploringShipLogbooks.basic_utils import encode_data_df
from exploringShipLogbooks.basic_utils import clean_data

from exploringShipLogbooks.config import *


# Load and clean data
### Load CLIWOC ship logs

In [2]:
cliwoc_data = extract_logbook_data('CLIWOC15.csv')

C:\Users\Emma\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3006: DtypeWarning: Columns (5,6,7,8,11,13,18,19,23,24,25,26,28,29,30,34,35,38,43,44,46,73,77,81,82,84,85,87,88,94,96,97,98,99,111,114,116,119,120,122,124,125,127,129,131,133,135,137,140) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


### Find definite slave data in CLIWOC data set
- These logs will be used to test the classifier

In [3]:
# extract logs that mention slaves
slave_mask = wc.count_key_words(cliwoc_data, text_columns, slave_words)

print('Found ', len(slave_mask[slave_mask==True]), ' logs that mention slaves')

Found  464  logs that mention slaves


### Clean CLIWOC data

In [4]:
# find indices of ship names that are "non-slave" ships before dropping ship name column
criteria = {'ShipName': non_slave_ships}
no_slave_mask = isolate_training_data(cliwoc_data, criteria)

In [5]:
# remove undesired columns
cliwoc_data = isolate_columns(cliwoc_data, desired_columns)

### Load Slave Voyages data

In [6]:
file_name = './exploringShipLogbooks/data/tastdb-exp-2010'
slave_voyage_logs = df = pd.read_pickle(file_name)

### Clean Slave voyages data

In [7]:
slave_voyage_desired_cols = ['portdep', 'portret', 'rig', 'national', 'yeardep']
slave_voyage_logs = isolate_columns(slave_voyage_logs, slave_voyage_desired_cols)

slave_voyage_logs.columns = ['Nationality', 'ShipType', 
                             'VoyageFrom', 'VoyageTo', 'Year']


# Join data sets

- Adding three indices to keep track of what dataset is which. We need to use this to index the classifier data later

In [8]:
cliwoc_data_indices_no_slaves = pd.DataFrame(0, index=cliwoc_data[~slave_mask].index, columns = ['data_indices'])
cliwoc_data_indices_slaves = pd.DataFrame(1, index=cliwoc_data[slave_mask].index, columns = ['data_indices'])
cliwoc_data_indices = pd.concat([cliwoc_data_indices_no_slaves, cliwoc_data_indices_slaves]).sort_index()
slave_data_indices = pd.DataFrame(2, index=(slave_voyage_logs.index + cliwoc_data.tail(1).index[0]), 
                                  columns = ['data_indices'])

- cliwoc_data (no slaves) = 0
- cliwoc_data (slaves)    = 1
- slave_data              = 2

In [9]:
indices = pd.concat([cliwoc_data_indices, slave_data_indices])

- okay, now back to your code. commented out the last lines since the above cells take care of indexing the data

In [10]:
all_data = pd.concat([cliwoc_data, slave_voyage_logs], ignore_index = True)
all_data = clean_data(all_data)


## further cleaning of data

- Nationality: 
    - british and great britain
    - french and france
    - spanish and spain
    - usa and american
    
- ShipName:
    - 9740 unique shipnames good column to filter!
    
- ShipType:
    - 118743 nan values for ship types (remove these columns?)
    - duplicate types in different languages, will this be a problem?
    
- VoyageFrom:
    - 1129 unique voyage starting points
    
- VoyageTo:
    - 1074 unique voyage ending points
    
- Year:
    - filter out the ships from the slave dataset that are before 1600?

- finds the rows where ShipType is equal to nan

In [ ]:
#ship_type_ind = ~(all_data['ShipType'] == 'nan')

- finds the empty strings in the ShipName column

In [ ]:
#empty_rows_ind = ~all_data['ShipName'].isin(all_data['ShipName'].value_counts().keys()[all_data['ShipName'].value_counts().keys() == ''])

- finds the ShipNames greater than some value count. Used 0 so the non-slave ship training data would be available...

In [ ]:
#ship_name_ind = all_data['ShipName'].isin(all_data['ShipName'].value_counts().keys()[all_data['ShipName'].value_counts()>0])

In [ ]:
#all_data = all_data[(empty_rows_ind & ship_name_ind & ship_type_ind)]
#indices = indices[(empty_rows_ind & ship_name_ind & ship_type_ind)]
#indices.index = range(len(indices))

In [ ]:
#all_data = all_data[ship_name_ind]
#indices = indices[ship_name_ind]
#indices.index = range(len(indices))

In [ ]:
from fuzzywuzzy import fuzz

In [ ]:
#fuzz.partial_ratio('dog','dog butt')

In [ ]:
def test(bob):
    print(bob)

In [ ]:
#all_data['Nationality'][0:1000].apply(lambda row: fuzz.partial_ratio(all_data['Nationality'][0:1000], row) >= 85)#, axis=1)

In [ ]:
def fuzzy_wuzzy_rows(name):
    matches = df.apply(lambda row: (fuzz.partial_ratio(row['Nationality'], name) >= 85), axis=1)
    return [i for i, x in enumerate(matches) if x]

In [ ]:
#df = pd.DataFrame( {'id':[1, 2, 3, 4, 5, 6], 'name':['dog', 'cat', 'mad cat', 'good dog', 'bad dog', 'chicken']})

In [ ]:
def func(name):
    matches = df.apply(lambda row: (fuzz.partial_ratio(row['Nationality'], name) >= 85), axis=1)
    return [i for i, x in enumerate(matches) if x]

In [ ]:
#all_data.apply(lambda row: func(row['Nationality']), axis=1)

# Encode data
- Must encode data before separating, otherwise values that do not occur in a subset will be encoded differently

In [11]:
all_data = encode_data_df(all_data)

# Extract training data, and create list of classes

In [12]:
unclassified_logs = all_data.iloc[cliwoc_data_indices_no_slaves.index]

validation_set_1 = all_data.iloc[cliwoc_data_indices_slaves.index]

# reserve first 20% of slave_voyage_logs as validation set
validation_set_2_indices = range(min(slave_data_indices.index),
                                 min(slave_data_indices.index) + round(len(slave_data_indices)*.2))
validation_set_2 = all_data.iloc[validation_set_2_indices]

training_logs_pos = all_data.iloc[slave_data_indices.index]
training_logs_pos = training_logs_pos.drop(validation_set_2_indices)

# note! This relies on cliwoc data being first in all_data
# could make more robust later
# TODO: check that this works!! should have some in first 1000?
training_logs_neg = all_data.iloc[no_slave_mask[no_slave_mask == True].index]

# cleanup
del all_data, cliwoc_data, slave_voyage_logs

In [18]:
# create list of classes for training data (0 is for non-slave, 1 is for slave)
# index matches training_data
classes = np.zeros(len(training_logs_neg))
classes = np.append(classes, np.ones(len(training_logs_pos)))

# joint training data
training_data = pd.concat([training_logs_neg, training_logs_pos], ignore_index = True)

# convert to numpy array
training_data = training_data.as_matrix()

array([[    0.,     0.,     0., ...,     0.,     0.,  1790.],
       [    0.,     0.,     0., ...,     0.,     0.,  1790.],
       [    0.,     0.,     0., ...,     0.,     0.,  1790.],
       ..., 
       [    0.,     0.,     0., ...,     0.,     0.,  1851.],
       [    0.,     0.,     0., ...,     0.,     0.,  1851.],
       [    0.,     0.,     0., ...,     0.,     0.,  1851.]])

# Fit training data to classifier
- **note!** first column of numpy array is index! do not include in classification!

In [30]:
classifier = MultinomialNB(alpha = 1.0, class_prior = None, fit_prior = True)
classifier.fit(training_data[::,1::], classes)

predictions = classifier.predict(unclassified_logs)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
predictions_count = collections.Counter(predictions)
print(predictions_count)

# Test classifier
- check if slave logs from cliwoc data were classified correctly (want mostly classified as 1)
- compare first column with slave_index


In [32]:
# sample to test
predictions = np.array([1,1,1,0,1,0,1])

In [43]:
counts = collections.Counter(predictions)
percent_correct = (counts[1]/(counts[1]+ counts[0]))* 100
print('Validation set was classified as a slave ship ',
     round(percent_correct,2), '% of the time')


Validation set was classified as a slave ship  71.43 % of the time
